# Nautilus Job Creation
This notebook will help you create a K8s Job on the Nautilus cluster

## Step 0: Prerequisites
* You have been added to a Nautilus namespace
* You have cloned down the SDSS2025 repository

## Step 1: Load the Template YAML File

We will be using a template of a Pod creation YAML to make this process easy:

```yaml
apiVersion: batch/v1

kind: Job

metadata:
  name: {{job_name}}

spec:
  ttlSecondsAfterFinished: 86400 # a day
  template:
    spec:
      automountServiceAccountToken: false
      affinity:
        nodeAffinity:
          requiredDuringSchedulingIgnoredDuringExecution:
            nodeSelectorTerms:
            - matchExpressions:
              - key: topology.kubernetes.io/region
                operator: In
                values:
                - us-central 
      containers:
        - name: pi
          image: perl:5.34
          command: ["perl", "-Mbignum=bpi", "-wle", "print bpi(2000)"]
          resources:
            limits:
              memory: 12Gi
              cpu: 2
            requests:
              memory: 10Gi
              cpu: 2
      restartPolicy: Never
  backoffLimit: 4

```

We are creating a Job in the central u.s. region.

Let's load our template using a template engine called `jinja2`:

In [ ]:
from jinja2 import Template

# read in the template
with open('../yaml/job_template.yml') as file_:
    template = Template(file_.read())

## Step 2: Render and Save the YAML File

Now, we can replace the name of the job we are going to create, and save the YAML File. 

Be sure to use something like your initials in the name of the job. Every namespace can only have a single job with a given name.

In [ ]:
# replace None 
JOB_NAME = input("Enter Job Name: ")

job_spec = template.render(job_name=JOB_NAME)

Let's view our rendered YAML:

In [ ]:
print(job_spec)

Finally, let's save the YAML to disk:

In [ ]:
with open("./job.yml", "w") as file:
    file.write(pod_spec)

## Step 3: Create the Job

Now, we need to invoke KubeCTL with our saved YAML file to create the Job:

In [1]:
! kubectl create -f ./job.yml

error: the path "./job.yml" does not exist


## Step 4: Verify Job & Pod Creation


Let's view all of the jobs in our namespace and ensure we find ours:

In [ ]:
! kubectl get jobs

Additionally, recall that jobs are nothing more than managed pods. Let's verify our Pod is running:

In [ ]:
! kubectl get pods

## Step 5: Job Running and Output 

We now wait for the job to finish. Once it has we can look at the output of our pod:

In [ ]:
! kubectl logs PODNAME

## Step 6: Deleting the Job

Once we've finished using the job, let's remove it:

In [ ]:
! kubectl delete job JOBNAME